<a href="https://colab.research.google.com/github/jatinarora-01/Spam-Mail-Detection-Model-using-BERT/blob/main/SpamMailUsingBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, GPT2LMHeadModel, GPT2Tokenizer
import os

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f"Using device: {device}")

Using device: cpu


In [ ]:
from google.colab import files

# Upload dataset
uploaded = files.upload()

Saving project_dataset.xlsx to project_dataset.xlsx


In [ ]:
# Load the dataset
file_path = "project_dataset.xlsx"
dataset = pd.read_excel(file_path)

In [ ]:
# Combine 'Subject' and 'Body' for text classification
dataset['Text'] = dataset['Subject'] + " " + dataset['Body']

# Encode labels
label_mapping = {"Ham": 0, "Spam": 1}
dataset['Label'] = dataset['Label'].map(label_mapping)

# Split into training and testing datasets
train_texts, test_texts, train_labels, test_labels = train_test_split(
    dataset['Text'].tolist(), dataset['Label'].tolist(), test_size=0.2, random_state=42
)

In [ ]:
# Load BERT tokenizer
tokenizer_bert = BertTokenizer.from_pretrained("bert-base-uncased")

# Tokenize dataset
train_encodings = tokenizer_bert(train_texts, truncation=True, padding=True, max_length=128)
test_encodings = tokenizer_bert(test_texts, truncation=True, padding=True, max_length=128)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
class EmailDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

In [ ]:
# Prepare datasets
train_dataset = EmailDataset(train_encodings, train_labels)
test_dataset = EmailDataset(test_encodings, test_labels)

In [ ]:
model_bert = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
model_bert.to(device)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",          # Output directory
    num_train_epochs=3,              # Number of training epochs
    per_device_train_batch_size=16,  # Batch size for training
    per_device_eval_batch_size=64,   # Batch size for evaluation
    warmup_steps=500,                # Warmup steps for learning rate scheduler
    weight_decay=0.01,               # Strength of weight decay
    logging_dir="./logs",            # Directory for storing logs
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Trainer(
    model=model_bert,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

# Train the model
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
1,0.715200,0.676157
2,0.435300,0.380869
3,0.220500,0.125817


TrainOutput(global_step=75, training_loss=0.5245165260632833, metrics={'train_runtime': 702.4377, 'train_samples_per_second': 1.696, 'train_steps_per_second': 0.107, 'total_flos': 12240830739600.0, 'train_loss': 0.5245165260632833, 'epoch': 3.0})

In [ ]:
# Save the fine-tuned model and tokenizer
output_dir = "./spam_detection_model"
os.makedirs(output_dir, exist_ok=True)
model_bert.save_pretrained(output_dir)
tokenizer_bert.save_pretrained(output_dir)
print(f"Model saved to {output_dir}")

Model saved to ./spam_detection_model


In [ ]:
from transformers import BertTokenizer, GPT2Tokenizer, GPT2LMHeadModel
import torch

# Load BERT tokenizer
tokenizer_bert = BertTokenizer.from_pretrained("bert-base-uncased")

# Load GPT2 tokenizer and model for content analysis and explanation generation
tokenizer_gpt = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer_gpt.pad_token = tokenizer_gpt.eos_token  # Set padding token as eos_token
tokenizer_gpt.add_special_tokens({'pad_token': '[PAD]'})  # Explicitly add a pad token

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

1

In [ ]:
model_gpt = GPT2LMHeadModel.from_pretrained("gpt2")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_gpt.to(device)

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [ ]:
from transformers import pipeline

# Load models for spam prediction and summarization
spam_classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
def predict_spam(email_subject, email_body):
    text = email_subject + " " + email_body
    # Here we classify the text as either spam or ham
    result = spam_classifier(text, candidate_labels=["spam", "ham"])
    return result["labels"][0]

In [ ]:
def generate_summary(email_subject, email_body):
    text = email_subject + " " + email_body
    input_length = len(text.split())

    # Set max_length dynamically based on input length and ensure it's smaller than the input
    max_length = min(input_length, 50)  # Don't exceed 50 tokens, which is a reasonable length
    summary = summarizer(text, max_length=max_length, min_length=5, do_sample=False, truncation=True)

    return summary[0]['summary_text']

In [ ]:
def generate_explanation(email_subject, email_body):
    text = email_subject + " " + email_body
    explanation_prompt = f"Why is the following email flagged as spam? Explain clearly in a few sentences.\n\n{email_subject}\n{email_body}\n\nExplanation:"

    explanation_model = pipeline("text-generation", model="gpt2")
    explanation = explanation_model(explanation_prompt, max_length=150, num_return_sequences=1)[0]['generated_text']

    # Post-process the explanation
    explanation = explanation.split("Explanation:")[-1].strip()
    return explanation

In [ ]:
# List of email subjects and bodies
emails = [
    {"subject": "Business Update", "body": "Please get reward by clicking this link."},
    {"subject": "IRCTC: Train Ticket Booking Confirmation", "body": "Your train ticket has been confirmed."},
    {"subject": "Business Update", "body": "Please find the minutes of today's meeting attached."},
]

# Loop through each email and predict if it's spam or ham
for email in emails:
    email_subject = email["subject"]
    email_body = email["body"]

    # Predict if the email is spam or ham
    prediction = predict_spam(email_subject, email_body)
    print(f"Subject: {email_subject}")
    print(f"Prediction: {prediction}")
    print("-" * 50)

Subject: Business Update
Prediction: spam
--------------------------------------------------
Subject: IRCTC: Train Ticket Booking Confirmation
Prediction: ham
--------------------------------------------------
Subject: Business Update
Prediction: ham
--------------------------------------------------
